In [1]:
import torch
from utils.aihub_kpop_dataset import KpopImageDatasetwT
from torch.utils.data import DataLoader
from model import PoseHRNet
model = PoseHRNet('w32', num_joints=29)

In [2]:
mydataset = KpopImageDatasetwT()

In [3]:
mydataset.__len__()

3447

In [4]:
x, y = mydataset.__getitem__(0)

/Users/junghyun/zerotohero/pytorch/HRNet/utils/aihub_kpop_dataset.py:42: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/runner/miniforge3/conda-bld/pytorch-recipe_1647804319176/work/torch/csrc/utils/tensor_new.cpp:210.)
  return torch.Tensor(empty_list)


In [5]:
x.shape, y.shape

(torch.Size([3, 256, 256]), torch.Size([29, 64, 64]))

In [6]:
mydataloader = DataLoader(dataset=mydataset, batch_size=16, shuffle=True)

In [7]:
device = 'cpu'

In [14]:
from utils.loss_posehrnet import JointsMSELoss

myloss = torch.nn.MSELoss()
myoptimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

AttributeError: module 'torch.optim' has no attribute 'adam'

In [13]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    
    device = 'cpu'
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 1 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [10]:
def test(dataloader, model, loss_fn):
    device = 'cpu'
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [11]:
for X, y in mydataloader:
    print(X.shape, y.shape)
    break

torch.Size([16, 3, 256, 256]) torch.Size([16, 29, 64, 64])


In [12]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(mydataloader, model, myloss, myoptimizer)
    test(mydataloader, model, myloss)
print("Done!")

Epoch 1
-------------------------------


TypeError: forward() missing 1 required positional argument: 'target_weight'